In [1]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration

# Import some basic libraries and functions for this tutorial.
import numpy as np
import os

from pydrake.common import temp_directory
from pydrake.geometry import SceneGraphConfig, StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer
# from manipulation.scenarios import SetColor

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
schunk_wsg50_model_url = (
    "package://drake_models/wsg_50_description/sdf/"
    "schunk_wsg_50_with_tip.sdf")

# Create a NEW model visualizer and add the robot gripper.
visualizer = ModelVisualizer(meshcat=meshcat)
grasp = visualizer.parser().AddModels(url=schunk_wsg50_model_url)[0]

In [4]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

grasp2 = parser.AddModelsFromUrl(
    "package://drake/examples/planar_gripper/planar_gripper.sdf"
)[0]
brick = parser.AddModelsFromUrl(
    "package://drake_models/manipulation_station/061_foam_brick.sdf"
)[0]
grasp = parser.AddModelsFromUrl(
    "package://drake_models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
)[0]
pregrasp = parser.AddModelsFromUrl(
    "package://drake_models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
)[0]

plant.Finalize()

In [5]:
print(grasp)
print(grasp2)

ModelInstanceIndex(4)
ModelInstanceIndex(2)


In [6]:
B_O = plant.GetBodyByName("base_link", brick)
B_Ggrasp = plant.GetBodyByName('body', grasp)
B_Gpregrasp = plant.GetBodyByName("body", pregrasp)

In [7]:
# Set the pregrasp to be green and slightly transparent.
inspector = scene_graph.model_inspector()

for body in [B_O, B_Ggrasp, B_Gpregrasp]:#, B_Gpregrasp]:
    AddFrameTriadIllustration(
        plant=plant,
        scene_graph=scene_graph,
        body=body,
        length=0.12
    )

In [8]:
meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

In [9]:
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)



In [10]:

# Get the current object, O, pose
world_body = plant.world_body()
X_WO = plant.EvalBodyPoseInWorld(plant_context, world_body)

p_GgraspO = [0, .12, 0]
R_GgraspO = RotationMatrix.MakeXRotation(np.pi / 2).multiply(RotationMatrix.MakeZRotation(np.pi / 2))

X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
X_OGgrasp = X_GgraspO.inverse()
X_WGgrasp = X_WO.multiply(X_OGgrasp)

# pregrasp is negative y in the gripper frame (see the figure!).
X_GgraspGpregrasp = RigidTransform([0, -0.2, 0])
X_WGpregrasp = X_WGgrasp @ X_GgraspGpregrasp

plant.SetFreeBodyPose(plant_context, B_Ggrasp, X_WGgrasp)
# Open the fingers, too.
plant.GetJointByName("left_finger_sliding_joint", grasp).set_translation(
    plant_context, -0.025
)
plant.GetJointByName("right_finger_sliding_joint", grasp).set_translation(
    plant_context, 0.025
)

plant.SetFreeBodyPose(plant_context, B_Gpregrasp, X_WGpregrasp)
# Open the fingers, too.
plant.GetJointByName("left_finger_sliding_joint", pregrasp).set_translation(
    plant_context, -0.054
)
plant.GetJointByName("right_finger_sliding_joint", pregrasp).set_translation(
    plant_context, 0.054
) 




<PrismaticJoint name='right_finger_sliding_joint' index=12 model_instance=5>

In [11]:
diagram.ForcedPublish(context)